# Machine Learning Engineer Nanodegree
## Capstone Proposal

Brian Palmer  
September 13, 2017

## Proposal

### Domain Background

Some of my primary areas of interest in machine learning are:

1. Using reinforcement learning to train AIs to solve complex tasks as they perform the tasks (as opposed to using supervised or unsupervised learning on pre-collected data sets).
2. Neural networks and AIs that can interpret and operate directly on high-dimensional sensor data, such as images or video streams.
3. Robotics and software for hardware interacting in real-world environments, such as in warehouses, factories, and self-driving cars/drones.

In recent years, much progress has been made in these areas. For instance, the [Deep Q Networks paper](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) outlines a deep convolutional neural network that directly reads video input, Atari video games, and learns discrete control policies to play many games at a human level. The subsequent [Deep Deterministic Policy Gradient paper](https://arxiv.org/pdf/1509.02971v2.pdf) solves continuous control problems (which are common in robotics) by building on the DQN work and adding an actor-critic architecture which can operate on high-dimensional continuous outputs. The [K-FAC paper](https://arxiv.org/pdf/1503.05671.pdf) outlines a method of using the natural gradient, based on the Fisher information matrix, to optimize a neural network in fewer timesteps.

Last year, the [Asynchronous Methods for Deep Reinforcement Learning paper](https://arxiv.org/pdf/1602.01783.pdf) outlined an asynchronous, parallel reinforcement learning agent that can outperform these earlier agents in a shorter period of time even on just a multi-core CPU, and supports continuous outputs as well as high-dimensional inputs. Later it was recognized that the asynchronous nature of the architecture was not important, and a deterministic version called Advantage Actor-Critic (A2C) has become popular.

### Problem Statement

For my capstone project, I propose tackling the [Natural Q-Learning](https://openai.com/requests-for-research/#natural-q-learning) Request For Research posted by OpenAI on their site. Their Requests For Research are a series of interesting open problems in Machine Learning/Deep Learning. Natural Q-Learning is the idea of extending Q-Learning to use Natural Gradient Descent for updates, instead of the typical Stochastic Gradient Descent approach, in the hopes that it will make Q-Learning more stable and converge to a solution in fewer timesteps.

Ultimately, the goal is to implement Q-Learning with natural gradients, using [Normalized Advantage Function](https://arxiv.org/pdf/1603.00748.pdf) or another method to support continuous action spaces. I will then train and test this model against the available discrete-action environments in the [OpenAI Gym](https://gym.openai.com/). Success is defined as creating an agent that can solve the various Gym envrionments without changes to architecture or hyperparameters. "Solved" means different things for different Gym environments, but typically it involves reaching a goal state in a certain number of timesteps, or avoiding a failure condition for a number of timesteps.

The theory is that using the natural gradient should allow the learning to be more stable and less reliant on randomized memory replay to avoid correlations from ordering in the training data, but as discussed in the Request For Research, this is unproven. Even if the experiment ends up being unsuccessful, I should generate useful information about how natural gradient updates behave in Q-Learning architectures.

### Datasets and Inputs

Because this is a Reinforcement Learning problem, rather than traditional Supervised or Unsupervised learning, the input states are generated during training as the simulations run, rather than being pre-gathered.

Some of these discrete action environments have low-dimensional state spaces consisting of a few (less than 100) input variables. These often correspond to position, velocity, angle, and other physical characteristics. Other environments use the raw pixel data as input, which is of much higher dimension (on the order of thousands of input variables). I will be operating against both types of environments.

All the gym envrionments I will be testing against take a discrete (integer) output as their action space. This often corresponds to a key press, such as a joystick button for the Atari environments.

### Solution Statement

The first step is for me to implement a basic but robust Deep Q-Learning agent. After that, I will investigate and implement natural gradient descent updates, replacing the stochastic gradient descent backpropagation.

After each step I will run the agent against a set of Gym environments appropriate to the agent at that stage, ensuring that the agent can solve each environment and gathering data in performance metrics including:

- Number of timesteps spent in learning before the environment is solved.
- Computational cost (measured via wall-clock time).
- RAM used for the replay memory.
- Stability of the agent: training it three times, and comparing how long it takes to solve the environment each time.

### Benchmark Model

Because the first step in my solution is implementing a basic Q-Learning agent, I can use that agent as my benchmark model. The goal with the natural gradient agent will be to improve on the basic agent's performance.

I will track the metrics listed above for the benchmark model as well as my extended natural gradient model, for evaluation.

### Evaluation Metrics

The metrics listed above will be used for ultimate evaluation of both the simple benchmark model and the natural gradient model.

The metrics can be ordered by relative importance:

1. Number of timestamps before a solution is reached, for each training of the agent. This is a simple counter provided by the Gym environment. The primary reason that this is so important isn't just because more timestamps means a longer training time. In robotics problems, it's very important to be efficient in the amount of training required, since training has real-world costs.
2. Relative stability of the learning. This can be estimated by measuring the variance in the first metric across all the trainings for the agent, for a given environment. In other words, given a series of training sessions $x$, with $x_i$ being the number of timestamps before the environment is solved for training session $i$, the learning stability can be estimated as $\frac{\sum{(x_i-\bar{x})^2}}{n-1}$.
3. Two computational metrics: wall clock time, and RAM used for storing replay memories.

### Project Design

I plan to implement my agent using Python 3 with [Keras](https://keras.io) using the [Tensorflow](https://www.tensorflow.org) back-end. There's a possibility that I may need to drop down and use Tensorflow directly to implement the natural gradient updates, it isn't clear to me yet whether the Keras API contract allows for everything I'll need.

The first step, the basic Deep Q-Learning agent, will follow the linked DQN paper closely. The algorithm can be summarized as:

Initialize replay memory deque $R$ to capacity $N$. Initialize action-value network $Q$ and target network $\hat{Q}$ with random weights. Then in informal notation, for each training step:

1. With probability `epsilon` select a random action $a_t$. Otherwise select the highest-rated action predicted by $Q$.
2. Execute the chosen action $a_t$ and observe reward $r_t$ and next state $s_{t+1}$.
3. Store $(s_t, a_t, r_t, s_{t+1})$ into $R$.
4. Sample a random subset of the memories stored in $R$, and for each memory:
    1. Estimate the reward $v$ as $r_t + \gamma * amax(\hat{Q}(s_{t+1}))$.
    2. Run backpropagation (fit) $Q$ using the estimated reward $v$ as the value for $a_t$ at $s_t$.
5. Every $C$ timesteps, reset $\hat{Q} = Q$.



#### Notes

Some of the gym environments have low-dimensional inputs, while others use raw pixels for input and thus have much higher dimensionality. While the ultimate goal is to have an architecture that can solve all the environments without any changes, I am making one concession: for pixel input the architecture will include some number of Convolutional Neural Network layers, while for low-dimensional environments these layers will be omitted.

I have already used Keras to implement a simple Deep Q-Learning agent that functions well enough to solve the basic CartPole gym environment. This gave me enough experience to be confident in selecting this as my project.

So far I have been running these agents locally, but I will likely spin up at least a couple AWS instances so that I can run the agents at a larger scale, especially since I am going to be training each agent on each environment multiple times, in order to gather the data on training stability. I have extensive AWS experience so I don't forsee any difficulties here.